In [3]:
import pandas as pd
from geopy.distance import geodesic
from geographiclib.geodesic import Geodesic
from datetime import datetime, timedelta

input_file = "./validation.csv"
df = pd.read_csv(input_file, delimiter=",", encoding="utf-8")

df['Дата вылета'] = pd.to_datetime(df['Дата вылета'])
df['Время вылета'] = pd.to_datetime(df['Время вылета'], format='%H:%M').dt.time
df['Время прилета'] = pd.to_datetime(df['Время прилета'], format='%H:%M').dt.time
df['Номер рейса'] = df['Номер рейса'].astype(str)
cdf = df.copy()

cdf['Год'] = cdf['Дата вылета'].dt.year
cdf['Месяц'] = cdf['Дата вылета'].dt.month
cdf['День'] = cdf['Дата вылета'].dt.day
cdf['День недели'] = cdf['Дата вылета'].dt.dayofweek

# https://www.kaggle.com/datasets/samvelkoch/global-airports-iata-icao-timezone-geo
adf = pd.read_csv("./airports.csv", delimiter=",", encoding="utf-8")

ADF_CODE = "IATA"
ADF_LATITUDE = "GeoPointLat"
ADF_LONGITUDE = "GeoPointLong"
ADF_CITY = "City_Name"
ADF_COUNTRY = "Country_Name"
ADF_COUNTRY_CODE = "Country_CodeA2"

cdf = cdf.merge(
    adf[[ADF_CODE, ADF_LATITUDE, ADF_LONGITUDE, ADF_CITY, ADF_COUNTRY, ADF_COUNTRY_CODE]],
    left_on='Аэропорт вылета',
    right_on=ADF_CODE,
    how='left'
)

cdf = cdf.rename(columns={
    ADF_LATITUDE: 'Широта аэропорта вылета',
    ADF_LONGITUDE: 'Долгота аэропорта вылета',
    ADF_CITY: 'Город аэропорта вылета',
    ADF_COUNTRY: 'Страна аэропорта вылета',
    ADF_COUNTRY_CODE: 'Код страны аэропорта вылета'
}).drop(ADF_CODE, axis=1)

cdf = cdf.merge(
    adf[[ADF_CODE, ADF_LATITUDE, ADF_LONGITUDE, ADF_CITY, ADF_COUNTRY, ADF_COUNTRY_CODE]],
    left_on='Аэропорт прилета',
    right_on=ADF_CODE,
    how='left'
)

cdf = cdf.rename(columns={
    ADF_LATITUDE: 'Широта аэропорта прилета',
    ADF_LONGITUDE: 'Долгота аэропорта прилета',
    ADF_CITY: 'Город аэропорта прилета',
    ADF_COUNTRY: 'Страна аэропорта прилета',
    ADF_COUNTRY_CODE: 'Код страны аэропорта прилета'
}).drop(ADF_CODE, axis=1)

cdf['Расстояние между аэропортами'] = cdf.apply(lambda row: geodesic(
    (row["Широта аэропорта вылета"], row["Долгота аэропорта вылета"]),
    (row["Широта аэропорта прилета"], row["Долгота аэропорта прилета"])
).kilometers, axis=1)

cdf['Международный рейс'] = cdf['Страна аэропорта вылета'] != cdf['Страна аэропорта прилета']
cdf['Вектор движения'] = cdf.apply(lambda row: Geodesic.WGS84.Inverse(
    row['Широта аэропорта вылета'], row['Долгота аэропорта вылета'],
    row['Широта аэропорта прилета'], row['Долгота аэропорта прилета'],
)['azi1'], axis=1)

gdf = pd.read_csv("./geonames-all-cities-with-a-population-1000.csv", delimiter=";", encoding="utf-8")
gdf['Alternate Names'] = gdf['Alternate Names'].astype(str)
gdf['Alternate Names'] = gdf['Alternate Names'].str.split(',')
gdf['all_names'] = gdf.apply(lambda row: list(set(
    [row['Name']] + 
    [row['ASCII Name']] + 
    ([] if row['Alternate Names'] == ['nan'] else row['Alternate Names'])
)), axis=1)
gdf = gdf.explode("all_names")
gdf['all_names'] = gdf['all_names'].str.lower()

# fixing kaliningrad-korolev issue
gdf = gdf.sort_values('Population', ascending=False)
gdf = gdf.drop_duplicates(subset=['all_names', 'Country Code'], keep='first')

rdf = cdf.copy()

rdf['city_dep'] = rdf["Город аэропорта вылета"].str.lower()
rdf['city_arr'] = rdf["Город аэропорта прилета"].str.lower()

rdf = rdf.merge(
    gdf[["all_names", "Population", "Timezone", "Country Code"]],
    left_on=['city_dep', "Код страны аэропорта вылета"],
    right_on=["all_names", "Country Code"],
    how='left'
)

rdf = rdf.rename(columns={
    "Population": "Популяция аэропорта вылета",
    "Timezone": "Часовой пояс аэропорта вылета"
}).drop(["all_names", "city_dep", "Country Code"], axis=1)

rdf = rdf.merge(
    gdf[["all_names", "Population", "Timezone", "Country Code"]],
    left_on=['city_arr', "Код страны аэропорта прилета"],
    right_on=["all_names", "Country Code"],
    how='left'
)

rdf = rdf.rename(columns={
    "Population": "Популяция аэропорта прилета",
    "Timezone": "Часовой пояс аэропорта прилета"
}).drop(["all_names", "city_arr", "Country Code"], axis=1)
def calculate_row_timedelta(row):
    date_dep = row["Дата вылета"]
    time_dep = row["Время вылета"]
    time_arr = row["Время прилета"]
    
    datetime_dep = datetime.combine(date_dep, time_dep)
    datetime_arr = datetime.combine(date_dep, time_arr)
    
    if datetime_arr < datetime_dep:
        datetime_arr += timedelta(days=1)
    
    return (datetime_arr - datetime_dep).total_seconds() / 60

rdf["Время в пути"] = rdf.apply(calculate_row_timedelta, axis=1)
rdf["Время вылета (минуты)"] = rdf["Время вылета"].apply(lambda x: x.hour*60 + x.minute)
rdf["Время прилета (минуты)"] = rdf["Время прилета"].apply(lambda x: x.hour*60 + x.minute)
from zoneinfo import ZoneInfo

def tz_difference(row) -> int:
    dt = datetime.now()
    
    # Локализуем дату в каждом часовом поясе
    dt1 = dt.astimezone(ZoneInfo(row['Часовой пояс аэропорта вылета']))
    dt2 = dt.astimezone(ZoneInfo(row['Часовой пояс аэропорта прилета']))
    
    # Разница в секундах → часы
    diff_hours = (dt1.utcoffset() - dt2.utcoffset()).total_seconds() / 3600
    return diff_hours

rdf["Разница часовых поясов"] = rdf.apply(tz_difference, axis=1)

# cabin_codes = ['C', 'W', 'Y']
# trdf_expanded = []
# for code in cabin_codes:
#     temp_df = trdf.copy()
#     temp_df['Код кабины'] = code
#     trdf_expanded.append(temp_df)
# trdf = pd.concat(trdf_expanded, ignore_index=True)

# izdf = pd.read_csv("hackathon_data_main.csv", delimiter=";", encoding="utf-8")
# df_expanded = trdf.copy()

# izdf['Номер рейса'] = izdf['Номер рейса'].astype(str)
# df_expanded['Номер рейса'] = df_expanded['Номер рейса'].astype(str)

# izdf['Доход пасс'] = pd.to_numeric(izdf['Доход пасс'].str.replace(',', '.'))
# izdf['LF Кабина'] = pd.to_numeric(izdf['LF Кабина'].str.replace(',', '.'))

# capacities_by_aircraft_cabin = izdf.groupby(['Тип ВС', 'Код кабины'])['Емкость кабины'].unique().reset_index()
# trdf_with_capacities = []
# for _, row in trdf.iterrows():
#     aircraft_type = row['Тип ВС']
#     cabin_code = row['Код кабины']
#     matching_capacities = capacities_by_aircraft_cabin[
#         (capacities_by_aircraft_cabin['Тип ВС'] == aircraft_type) & 
#         (capacities_by_aircraft_cabin['Код кабины'] == cabin_code)
#     ]
#     if not matching_capacities.empty:
#         capacities = matching_capacities.iloc[0]['Емкость кабины']
#         for capacity in capacities:
#             new_row = row.copy()
#             new_row['Емкость кабины'] = capacity
#             trdf_with_capacities.append(new_row)
# trdf = pd.DataFrame(trdf_with_capacities)

# trdf['Конфигурация'] = trdf['Тип ВС'].astype(str) + '_' + trdf['Емкость кабины'].astype(str) + '_' + trdf['Код кабины'].astype(str)
# unique_configs = trdf['Конфигурация'].unique()
# config_id_map = {config: idx for idx, config in enumerate(unique_configs)}
# trdf['ID конфигурации'] = trdf['Конфигурация'].map(config_id_map)
# trdf = trdf.drop('Конфигурация', axis=1)

# trdf
rdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Год,Месяц,День,...,Международный рейс,Вектор движения,Популяция аэропорта вылета,Часовой пояс аэропорта вылета,Популяция аэропорта прилета,Часовой пояс аэропорта прилета,Время в пути,Время вылета (минуты),Время прилета (минуты),Разница часовых поясов
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,False,88.151182,10381222,Europe/Moscow,1243500,Europe/Moscow,100.0,550,650,0.0
1,2025-06-29,6321,LED,KGD,14:50:00,15:45:00,SU9,2025,6,29,...,False,-129.051777,5351935,Europe/Moscow,475056,Europe/Kaliningrad,55.0,890,945,1.0
2,2025-07-02,1246,SVO,REN,17:50:00,22:00:00,320,2025,7,2,...,False,103.884623,10381222,Europe/Moscow,564773,Asia/Yekaterinburg,250.0,1070,1320,-2.0
3,2025-07-02,1713,KHV,SVO,15:45:00,16:45:00,77W,2025,7,2,...,False,-42.492703,618150,Asia/Vladivostok,10381222,Europe/Moscow,60.0,945,1005,7.0
4,2025-06-30,2143,AYT,SVO,14:20:00,19:10:00,32B,2025,6,30,...,True,11.132237,1344000,Europe/Istanbul,10381222,Europe/Moscow,290.0,860,1150,0.0
5,2025-06-05,209,PVG,SVO,11:40:00,16:15:00,77W,2025,6,5,...,True,-39.334087,22315474,Asia/Shanghai,10381222,Europe/Moscow,275.0,700,975,5.0
6,2025-06-19,1009,KGD,SVO,03:00:00,06:30:00,32B,2025,6,19,...,False,76.692545,475056,Europe/Kaliningrad,10381222,Europe/Moscow,210.0,180,390,-1.0
7,2025-06-27,526,SVO,DXB,11:25:00,18:20:00,77W,2025,6,27,...,True,149.482012,10381222,Europe/Moscow,3478300,Asia/Dubai,415.0,685,1100,-1.0
8,2025-06-23,1018,SVO,KGD,11:50:00,13:35:00,32B,2025,6,23,...,False,-89.422018,10381222,Europe/Moscow,475056,Europe/Kaliningrad,105.0,710,815,1.0
9,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,True,-164.004502,10381222,Europe/Moscow,1344000,Europe/Istanbul,285.0,485,770,0.0


In [72]:
wdf = rdf.copy()

zdf: pd.DataFrame = pd.read_pickle("./prepared-dataset.pkl")
zdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Median Бронирования по кабинам (ВС),IQR Бронирования по кабинам (ВС),Min Пассажиры (Рейс),Max Пассажиры (Рейс),Median Пассажиры (Рейс),IQR Пассажиры (Рейс),Min Пассажиры (ВС),Max Пассажиры (ВС),Median Пассажиры (ВС),IQR Пассажиры (ВС)
0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,32B,...,164.0,24.00,1.0,181.00,106.0,148.00,49.56,184.00,166.0,24.0
1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,333,...,11.0,10.00,1.0,80.00,14.0,26.00,2.00,25.00,11.0,10.0
2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,77W,...,221.0,148.25,2.0,268.00,19.0,123.50,65.96,395.68,231.5,145.5
3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,77W,...,15.0,11.00,2.0,268.00,19.0,123.50,1.00,28.00,15.0,11.0
4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,77W,...,14.0,11.00,2.0,268.00,19.0,123.50,3.00,27.00,14.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,32B,...,17.0,18.00,1.0,187.66,23.5,160.25,2.00,30.00,17.5,18.0
19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,32B,...,147.0,13.00,1.0,187.66,23.5,160.25,131.00,177.24,147.0,13.0
19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,32B,...,7.0,6.00,4.0,174.00,16.0,131.00,1.00,16.00,7.0,6.0
19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,32B,...,147.0,13.00,4.0,174.00,16.0,131.00,131.00,177.24,147.0,13.0


In [73]:
wdf = wdf.merge(
    zdf.drop_duplicates(subset=['Номер рейса'])[[
        'Номер рейса',
        'Min Доход пасс (Рейс)',
        'Max Доход пасс (Рейс)',
        'Median Доход пасс (Рейс)',
        'IQR Доход пасс (Рейс)',
        'Min LF Кабина (Рейс)',
        'Max LF Кабина (Рейс)',
        'Median LF Кабина (Рейс)',
        'IQR LF Кабина (Рейс)',
        'Min Пассажиры (Рейс)',
        'Max Пассажиры (Рейс)',
        'Median Пассажиры (Рейс)',
        'IQR Пассажиры (Рейс)',
        'Min Бронирования по кабинам (Рейс)',
        'Max Бронирования по кабинам (Рейс)',
        'Median Бронирования по кабинам (Рейс)',
        'IQR Бронирования по кабинам (Рейс)'
    ]],
    on='Номер рейса',
    how='left'
)
wdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Год,Месяц,День,...,Median LF Кабина (Рейс),IQR LF Кабина (Рейс),Min Пассажиры (Рейс),Max Пассажиры (Рейс),Median Пассажиры (Рейс),IQR Пассажиры (Рейс),Min Бронирования по кабинам (Рейс),Max Бронирования по кабинам (Рейс),Median Бронирования по кабинам (Рейс),IQR Бронирования по кабинам (Рейс)
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,0.80000,0.611700,2.0,178.00,77.0,147.00,1.0,177.00,77.0,146.00
1,2025-06-29,6321,LED,KGD,14:50:00,15:45:00,SU9,2025,6,29,...,0.85545,0.368125,1.0,119.10,80.5,88.25,1.0,119.44,79.0,88.00
2,2025-07-02,1246,SVO,REN,17:50:00,22:00:00,320,2025,7,2,...,0.84720,0.720000,1.0,180.00,123.0,147.00,1.0,180.00,122.0,147.00
3,2025-07-02,1713,KHV,SVO,15:45:00,16:45:00,77W,2025,7,2,...,0.82870,0.500000,3.9,377.00,24.0,262.00,3.0,371.00,24.0,257.75
4,2025-06-30,2143,AYT,SVO,14:20:00,19:10:00,32B,2025,6,30,...,0.43615,0.348675,3.0,183.98,21.0,48.25,3.0,178.24,20.5,45.00
5,2025-06-05,209,PVG,SVO,11:40:00,16:15:00,77W,2025,6,5,...,0.76670,0.391600,4.0,357.32,30.0,235.00,4.0,352.76,30.0,235.00
6,2025-06-19,1009,KGD,SVO,03:00:00,06:30:00,32B,2025,6,19,...,0.81250,0.672900,1.0,187.66,23.5,160.25,1.0,187.66,24.0,159.25
7,2025-06-27,526,SVO,DXB,11:25:00,18:20:00,77W,2025,6,27,...,0.56025,0.358775,6.0,248.00,30.0,64.00,6.0,245.98,29.5,63.00
8,2025-06-23,1018,SVO,KGD,11:50:00,13:35:00,32B,2025,6,23,...,0.86210,0.450500,4.0,174.00,16.0,131.00,3.0,173.56,16.0,132.00
9,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,0.43330,0.266000,3.0,161.48,18.0,53.00,3.0,159.64,18.0,51.00


In [105]:
from collections import defaultdict
flights = defaultdict(list)
for _, row in zdf.iterrows():
    flight_id = (
        row["Номер рейса"] + "_" +
        row["Аэропорт вылета"] + "_" +
        str(row["День"]) + "_" +
        str(row["Месяц"]) + "_" +
        str(row["Год"]) + "_" +
        str(row["Время вылета"]) + "_" +
        str(row["Тип ВС"])
    )
    flights[flight_id].append((
        row['Код кабины'],
        row["Емкость кабины"],
    ))
configs = defaultdict(set)
for k, v in flights.items():
    configs[k.split('_')[-1]].add(",".join([f"{a}_{b}" for a,b in sorted(v)]))
cfg_rows = []
for _, row in wdf.iterrows():
    for cfg in configs[row['Тип ВС']]:
        for cabin_var in cfg.split(','):
            cfg_rows.append({
                **row,
                'Код кабины': cabin_var.split('_')[0],
                'Емкость кабины': int(cabin_var.split('_')[1]),
                'row-cfg-id': (
                    row["Номер рейса"] + "_" +
                    row["Аэропорт вылета"] + "_" +
                    str(row["День"]) + "_" +
                    str(row["Месяц"]) + "_" +
                    str(row["Год"]) + "_" +
                    str(row["Время вылета"]) + "_" +
                    str(row["Тип ВС"]) + "_" +
                    cfg
                ),
                'row-id': (
                    row["Номер рейса"] + "_" +
                    row["Аэропорт вылета"] + "_" +
                    str(row["День"]) + "_" +
                    str(row["Месяц"]) + "_" +
                    str(row["Год"]) + "_" +
                    str(row["Время вылета"]) + "_" +
                    str(row["Тип ВС"])
                )
            })
mdf = pd.DataFrame(cfg_rows)
mdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Год,Месяц,День,...,Median Пассажиры (Рейс),IQR Пассажиры (Рейс),Min Бронирования по кабинам (Рейс),Max Бронирования по кабинам (Рейс),Median Бронирования по кабинам (Рейс),IQR Бронирования по кабинам (Рейс),Код кабины,Емкость кабины,row-cfg-id,row-id
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,77.0,147.0,1.0,177.00,77.0,146.0,C,16,"1190_SVO_16_6_2025_09:10:00_32B_C_16,W_29,Y_138",1190_SVO_16_6_2025_09:10:00_32B
1,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,77.0,147.0,1.0,177.00,77.0,146.0,W,29,"1190_SVO_16_6_2025_09:10:00_32B_C_16,W_29,Y_138",1190_SVO_16_6_2025_09:10:00_32B
2,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,77.0,147.0,1.0,177.00,77.0,146.0,Y,138,"1190_SVO_16_6_2025_09:10:00_32B_C_16,W_29,Y_138",1190_SVO_16_6_2025_09:10:00_32B
3,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,77.0,147.0,1.0,177.00,77.0,146.0,W,29,"1190_SVO_16_6_2025_09:10:00_32B_W_29,Y_138",1190_SVO_16_6_2025_09:10:00_32B
4,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,77.0,147.0,1.0,177.00,77.0,146.0,Y,138,"1190_SVO_16_6_2025_09:10:00_32B_W_29,Y_138",1190_SVO_16_6_2025_09:10:00_32B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,18.0,53.0,3.0,159.64,18.0,51.0,Y,264,2142_SVO_14_6_2025_08:05:00_359_Y_264,2142_SVO_14_6_2025_08:05:00_359
107,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,18.0,53.0,3.0,159.64,18.0,51.0,W,24,"2142_SVO_14_6_2025_08:05:00_359_W_24,Y_264",2142_SVO_14_6_2025_08:05:00_359
108,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,18.0,53.0,3.0,159.64,18.0,51.0,Y,264,"2142_SVO_14_6_2025_08:05:00_359_W_24,Y_264",2142_SVO_14_6_2025_08:05:00_359
109,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,18.0,53.0,3.0,159.64,18.0,51.0,C,28,"2142_SVO_14_6_2025_08:05:00_359_C_28,Y_264",2142_SVO_14_6_2025_08:05:00_359


In [106]:
mdf = mdf.merge(
    zdf.drop_duplicates(subset=['Тип ВС', 'Код кабины', 'Емкость кабины'])[[
        'Тип ВС', 'Код кабины', 'Емкость кабины',
        'Min Доход пасс (ВС)',
        'Max Доход пасс (ВС)',
        'Median Доход пасс (ВС)',
        'IQR Доход пасс (ВС)',
        'Min LF Кабина (ВС)',
        'Max LF Кабина (ВС)',
        'Median LF Кабина (ВС)',
        'IQR LF Кабина (ВС)',
        'Min Пассажиры (ВС)',
        'Max Пассажиры (ВС)',
        'Median Пассажиры (ВС)',
        'IQR Пассажиры (ВС)',
        'Min Бронирования по кабинам (ВС)',
        'Max Бронирования по кабинам (ВС)',
        'Median Бронирования по кабинам (ВС)',
        'IQR Бронирования по кабинам (ВС)'
    ]],
    on=['Тип ВС', 'Код кабины', 'Емкость кабины'],
    how='left'
)
mdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Год,Месяц,День,...,Median LF Кабина (ВС),IQR LF Кабина (ВС),Min Пассажиры (ВС),Max Пассажиры (ВС),Median Пассажиры (ВС),IQR Пассажиры (ВС),Min Бронирования по кабинам (ВС),Max Бронирования по кабинам (ВС),Median Бронирования по кабинам (ВС),IQR Бронирования по кабинам (ВС)
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,0.4375,0.37500,1.0,16.00,7.0,6.00,1.0,15.00,7.0,6.00
1,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,0.5862,0.62070,2.0,30.00,17.5,18.00,2.0,29.00,17.0,18.00
2,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,1.0652,0.09420,131.0,177.24,147.0,13.00,131.0,175.00,147.0,13.00
3,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,0.5862,0.62070,2.0,30.00,17.5,18.00,2.0,29.00,17.0,18.00
4,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,1.0652,0.09420,131.0,177.24,147.0,13.00,131.0,175.00,147.0,13.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,0.5152,0.56345,35.0,289.86,137.0,153.00,35.0,287.86,136.0,148.75
107,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,0.4167,0.54160,1.0,24.00,10.0,13.00,1.0,24.00,10.0,13.00
108,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,0.5152,0.56345,35.0,289.86,137.0,153.00,35.0,287.86,136.0,148.75
109,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,0.4643,0.39290,2.0,26.00,13.0,10.75,2.0,26.00,13.0,11.00


In [107]:
from catboost import CatBoostRegressor, Pool

CAT_FEATURES = ["Аэропорт вылета", "Аэропорт прилета", "Тип ВС",
                "Город аэропорта вылета", "Город аэропорта прилета",
                "Страна аэропорта вылета", "Страна аэропорта прилета",
                "Часовой пояс аэропорта вылета", "Часовой пояс аэропорта прилета",
                "Код кабины"]

small_context_load = CatBoostRegressor()
small_context_load.load_model('./models/small-context-load.cbm')

small_context_revenue = CatBoostRegressor()
small_context_revenue.load_model('./models/small-context-revenue.cbm')

In [108]:
mdf['Бронирования по кабинам'] = small_context_load.predict(Pool(mdf[small_context_load.feature_names_], cat_features=CAT_FEATURES))
mdf['Доход пасс'] = small_context_revenue.predict(Pool(mdf[small_context_load.feature_names_], cat_features=CAT_FEATURES))
mdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Год,Месяц,День,...,Min Пассажиры (ВС),Max Пассажиры (ВС),Median Пассажиры (ВС),IQR Пассажиры (ВС),Min Бронирования по кабинам (ВС),Max Бронирования по кабинам (ВС),Median Бронирования по кабинам (ВС),IQR Бронирования по кабинам (ВС),Бронирования по кабинам,Доход пасс
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,1.0,16.00,7.0,6.00,1.0,15.00,7.0,6.00,6.338302,862.669492
1,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,2.0,30.00,17.5,18.00,2.0,29.00,17.0,18.00,13.024113,473.041019
2,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,131.0,177.24,147.0,13.00,131.0,175.00,147.0,13.00,154.134165,5605.406457
3,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,2.0,30.00,17.5,18.00,2.0,29.00,17.0,18.00,13.024113,473.041019
4,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,2025,6,16,...,131.0,177.24,147.0,13.00,131.0,175.00,147.0,13.00,154.134165,5605.406457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,35.0,289.86,137.0,153.00,35.0,287.86,136.0,148.75,117.710481,18445.764027
107,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,1.0,24.00,10.0,13.00,1.0,24.00,10.0,13.00,11.619624,2980.238241
108,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,35.0,289.86,137.0,153.00,35.0,287.86,136.0,148.75,117.710481,18445.764027
109,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,2025,6,14,...,2.0,26.00,13.0,10.75,2.0,26.00,13.0,11.00,17.270449,7276.781424


In [117]:
vdf = mdf.copy()
vdf['Доход'] = vdf.groupby(['row-id', 'row-cfg-id'])['Доход пасс'].transform('sum')
vdf['Бронирования'] = vdf.groupby(['row-id', 'row-cfg-id'])['Бронирования по кабинам'].transform('sum')
vdf = vdf.loc[vdf.groupby(['row-id'])['Доход'].transform('max') == vdf['Доход']]
vdf = vdf['Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС'.split(',') + ["Доход", 'Бронирования']]
vdf = vdf.drop_duplicates().reset_index(drop=True)
vdf

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Тип ВС,Доход,Бронирования
0,2025-06-16,1190,SVO,KZN,09:10:00,10:50:00,32B,7548.156070,167.307905
1,2025-06-29,6321,LED,KGD,14:50:00,15:45:00,SU9,9451.264216,115.768697
2,2025-07-02,1246,SVO,REN,17:50:00,22:00:00,320,10547.284299,167.637279
3,2025-07-02,1713,KHV,SVO,15:45:00,16:45:00,77W,55358.324895,417.915850
4,2025-06-30,2143,AYT,SVO,14:20:00,19:10:00,32B,14887.118129,101.188155
5,2025-06-05,209,PVG,SVO,11:40:00,16:15:00,77W,93570.715133,252.160543
6,2025-06-19,1009,KGD,SVO,03:00:00,06:30:00,32B,7385.264412,187.033241
7,2025-06-27,526,SVO,DXB,11:25:00,18:20:00,77W,59728.107765,211.648327
8,2025-06-23,1018,SVO,KGD,11:50:00,13:35:00,32B,9060.070975,164.988141
9,2025-06-14,2142,SVO,AYT,08:05:00,12:50:00,359,28702.783691,146.600554


In [118]:
vdf.to_csv("./validation-predicted.csv", sep=";", encoding="utf-8", index=False)